九年前的画面，依旧清晰无比。那是一个清晨，初升的太阳，朗朗的书声，清扫着周遭的寒气。教学楼后罚球线上的我，拿着还不怎么会拍的球，投进了人生中的第一个空心，清脆刷网声响起的那一刻，我知道，篮球，这个我曾经最害怕的运动，将完成角色的反转，成为我的一生所爱，伴我走过每一个春夏与秋冬，直到生命的尽头。

加入“运筹OR帷幄”公众号数据科学版块一个多月了，我一直在构思自己的第一篇原创文章应该写点什么，我是应该写篇技术性的文章，还是写篇分析性的文章，想来想去我发现我想得太多，反而束缚了我的手脚。于是我决定放飞自我，想到什么写什么，写成散文，情真意切，未尝不可。就这样，原本的标题“投篮数据分析”摇身一变，成了现在的模样。

最近，火箭队总经理莫雷的言论，NBA 总裁肖华的补刀，将 NBA 推上了舆论的风口浪尖。NBA 与中国关系的乌云何时散去还未可知，但我希望，也相信，万里晴空终会到来。

美国，作为篮球运动的诞生地，她的职业联赛，依旧代表着世界篮球发展的最高水平。在 NBA 官方统计网站上，不仅提供了大量的汇总数据，还提供了大量的原始数据。在那里，我们可以找到每场比赛的详细过程、可以找到每个球员每次出手的文字与视频记录、也可以找到每个球员的详细个人信息。当拿着 CBA 的统计网站和 NBA 对比一下，你能感受到，那扑面而来的差距。

接下来，就让我们来看一下 NBA 官方提供的球员投篮数据，看看针对这份数据，我们可以做哪些分析，获得哪些信息。

# 1 数据的获取

In [ ]:
import requests
import json
import pandas as pd

# 获取球员 ID 信息
url = "https://stats.nba.com/stats/commonallplayers?"
params = {
    "LeagueID": "00",
    "Season": "2019",
    "IsOnlyCurrentSeason": 0
}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
           + 'AppleWebKit/537.36 (KHTML, like Gecko) '
           + 'Chrome/77.0.3865.90 Safari/537.36'}
try:
    idInfo = requests.get(url, params=params, headers=headers).json()["resultSets"][0]
except Exception as e:
    print("\n错误：球员 ID 信息获取失败，请确认网络连接正常后重启程序！")
    exit()
else:
    print("\n成功：球员 ID 信息获取成功\n")
    idInfo = pd.DataFrame(idInfo["rowSet"], columns=idInfo["headers"])
    playerIDList = idInfo["PERSON_ID"].tolist()
    
# 获取球员常规赛投篮数据
shotDF, errorList, emptyList = pd.DataFrame(), [], []
for i, playerID in enumerate(playerIDList):
    url = 'https://stats.nba.com/stats/shotchartdetail?'
    params = {
        "SeasonType": "Regular Season",
        "TeamID": 0,
        "PlayerID": playerID,
        "PlayerPosition": '',
        "GameID": '',
        "Outcome": '',
        "Location": '',
        "Month": 0,
        "SeasonSegment": '',
        "DateFrom": '',
        "DateTo": '',
        "OpponentTeamID": 0,
        "VsConference": '',
        "VsDivision": '',
        "RookieYear": '',
        "GameSegment": '',
        "Period": 0,
        "LastNGames": 0,
        "ContextMeasure": "FGA",
    }
    try:
        shotDFSec = requests.get(url, params=params, headers=headers).json()["resultSets"][0]
    except Exception as e:
        errorList.append(playerID)
        print('错误：第{0}个球员（ID:{1}）数据获取失败'.format(i+1, playerID))
    else:
        print('成功：第{0}个球员（ID:{1}）数据获取成功'.format(i+1, playerID))
        if shotDFSec["rowSet"] != []:
            shotDFSec = pd.DataFrame(shotDFSec["rowSet"], 
                                     columns=shotDFSec["headers"])
            shotDF = shotDF.append(shotDFSec)
        else:
            emptyList.append(playerID)
            print('警告：第{0}个球员（ID:{1}）数据为空'.format(i+1, playerID))
    print('\n')

if emptyList != []:
    print('警告：以下球员 ID 数据为空\n{0}\n'.format(emptyList))
    
if errorList != []:
    print('错误：以下球员 ID 数据获取失败\n{0}\n'.format(errorList))

# 将数据保存到外部文件
shotDF.to_csv('F:/web_crawler_results/NBA/shotInfo.csv')

# 2 数据概览

In [44]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def convert_df(df: pd.DataFrame, deep_copy: bool = True) -> pd.DataFrame:
    """Automatically converts columns that are worth stored as
    ``categorical`` dtype.
    Parameters
    ----------
    df: pd.DataFrame
        Data frame to convert.
    deep_copy: bool
        Whether or not to perform a deep copy of the original data frame.
    Returns
    -------
    pd.DataFrame
        Optimized copy of the input data frame.
    """
    return df.copy(deep=deep_copy).astype({
        col: 'category' for col in df.columns
        if df[col].nunique() / df[col].shape[0] < 0.5})


# 查看数据框前 5 行
shotDF = pd.read_csv('f:/web_crawler_results/NBA/shotInfo2.csv').iloc[:,1:].pipe(convert_df)
shotDF.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,20000054,369,51,Mahmoud Abdul-Rauf,1610612763,Vancouver Grizzlies,3,0,38,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,20,64,200,1,0,20001106,VAN,ATL
1,Shot Chart Detail,20000143,131,51,Mahmoud Abdul-Rauf,1610612763,Vancouver Grizzlies,2,9,22,Made Shot,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,9,1,97,1,1,20001118,VAN,DAL
2,Shot Chart Detail,20000174,313,51,Mahmoud Abdul-Rauf,1610612763,Vancouver Grizzlies,3,6,42,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,18,163,82,1,0,20001124,DET,VAN
3,Shot Chart Detail,20000174,352,51,Mahmoud Abdul-Rauf,1610612763,Vancouver Grizzlies,3,2,42,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-111,127,1,0,20001124,DET,VAN
4,Shot Chart Detail,20000174,360,51,Mahmoud Abdul-Rauf,1610612763,Vancouver Grizzlies,3,2,18,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,15,150,49,1,0,20001124,DET,VAN


In [7]:
shotDF.dtypes

GRID_TYPE              category
GAME_ID                category
GAME_EVENT_ID          category
PLAYER_ID              category
PLAYER_NAME            category
TEAM_ID                category
TEAM_NAME              category
PERIOD                 category
MINUTES_REMAINING      category
SECONDS_REMAINING      category
EVENT_TYPE             category
ACTION_TYPE            category
SHOT_TYPE              category
SHOT_ZONE_BASIC        category
SHOT_ZONE_AREA         category
SHOT_ZONE_RANGE        category
SHOT_DISTANCE          category
LOC_X                  category
LOC_Y                  category
SHOT_ATTEMPTED_FLAG    category
SHOT_MADE_FLAG         category
GAME_DATE              category
HTM                    category
VTM                    category
dtype: object

In [18]:
import numpy as np
nadict = {}
for col in shotDF.columns:
    shotDFcol = shotDF[col]
    if any(shotDFcol.isnull()):
        indexList = shotDFcol[shotDFcol.isnull()].index.tolist()
        nadict[col] = indexList
        print('{0} 列存在缺失，所在行索引为 {1}'.format(col, indexList))

nadict
#shotDF.at[2870012, 'SHOT_TYPE'] = '2PT Field Goal'

PLAYER_NAME 列存在缺失，所在行索引为 [805770, 3656898, 3656899, 3656900, 3656901]
SHOT_TYPE 列存在缺失，所在行索引为 [2870012]


{'PLAYER_NAME': [805770, 3656898, 3656899, 3656900, 3656901],
 'SHOT_TYPE': [2870012]}

In [30]:
idInfo = shotDF[['PLAYER_ID', 'PLAYER_NAME']].drop_duplicates().dropna(how='any')
for i in nadict['PLAYER_NAME']:
    shotDF.at[i, 'PLAYER_NAME'] = idInfo[idInfo.PLAYER_ID==shotDF.at[i, 'PLAYER_ID']].PLAYER_NAME

,PLAYER_ID,PLAYER_NAME
0,51,Mahmoud Abdul-Rauf
1443,1505,Tariq Abdul-Wahad
3169,949,Shareef Abdur-Rahim
14685,203518,Alex Abrines
15473,101165,Alex Acker
15565,203112,Quincy Acy
16878,200801,Hassan Adams
17026,1629121,Jaylen Adams
17136,203919,Jordan Adams
17228,203500,Steven Adams


In [17]:
playerID_na = shotDF.iloc[[805770, 3656898, 3656899, 3656900, 3656901], 3].unique().tolist()
playerID_na
for i in playerID_na:
    print(shotDF[shotDF.PLAYER_ID==i]['PLAYER_NAME'].unique())

[Bimbo Coles, NaN]
Categories (1, object): [Bimbo Coles]
[Lionel Simmons, NaN]
Categories (1, object): [Lionel Simmons]


In [9]:
shotDF.PERIOD.unique()

[3, 2, 4, 1, 5, 6, 7, 8]
Categories (8, int64): [3, 2, 4, 1, 5, 6, 7, 8]

In [10]:
shotDF.ACTION_TYPE.unique()

[Jump Shot, Running Jump Shot, Layup Shot, Turnaround Jump Shot, Driving Layup Shot, ..., Running Alley Oop Layup Shot, Step Back Bank Jump Shot, Running Reverse Dunk Shot, Turnaround Finger Roll Shot, Putback Reverse Dunk Shot]
Length: 70
Categories (70, object): [Jump Shot, Running Jump Shot, Layup Shot, Turnaround Jump Shot, ..., Step Back Bank Jump Shot, Running Reverse Dunk Shot, Turnaround Finger Roll Shot, Putback Reverse Dunk Shot]

In [11]:
shotDF.SHOT_TYPE.unique()

[2PT Field Goal, 3PT Field Goal, NaN]
Categories (2, object): [2PT Field Goal, 3PT Field Goal]

In [12]:
shotDF.SHOT_ZONE_AREA.unique()

[Center(C), Right Side(R), Left Side Center(LC), Right Side Center(RC), Left Side(L), Back Court(BC)]
Categories (6, object): [Center(C), Right Side(R), Left Side Center(LC), Right Side Center(RC), Left Side(L), Back Court(BC)]

In [13]:
shotDF.SHOT_ZONE_RANGE.unique()

[16-24 ft., 8-16 ft., Less Than 8 ft., 24+ ft., Back Court Shot]
Categories (5, object): [16-24 ft., 8-16 ft., Less Than 8 ft., 24+ ft., Back Court Shot]

In [14]:
shotDF.SHOT_DISTANCE.unique()

[20, 9, 18, 16, 15, ..., 84, 72, 87, 89, 88]
Length: 90
Categories (90, int64): [20, 9, 18, 16, ..., 72, 87, 89, 88]

In [15]:
shotDF.HTM.unique()

[VAN, DET, NYK, PHX, DEN, ..., NOK, CHA, OKC, NOP, BKN]
Length: 36
Categories (36, object): [VAN, DET, NYK, PHX, ..., CHA, OKC, NOP, BKN]

In [16]:
shotDF.VTM.unique()

[ATL, DAL, VAN, HOU, GSW, ..., CHA, NOK, OKC, BKN, NOP]
Length: 36
Categories (36, object): [ATL, DAL, VAN, HOU, ..., NOK, OKC, BKN, NOP]

In [18]:
shotDF.LOC_X.unique()

[64, 1, 163, -111, 150, ..., -248, -250, -247, -249, 249]
Length: 501
Categories (501, int64): [64, 1, 163, -111, ..., -250, -247, -249, 249]

In [17]:
shotDF.LOC_Y.unique()

[200, 97, 82, 127, 49, ..., 860, 881, 862, 849, 871]
Length: 917
Categories (917, int64): [200, 97, 82, 127, ..., 881, 862, 849, 871]

## NBA 球员投篮选择的变化

In [19]:
shotDF.GAME_DATE.unique()

[20001106, 20001118, 20001124, 20001127, 20001211, ..., 20150212, 20061225, 19961225, 20031102, 19971231]
Length: 3646
Categories (3646, int64): [20001106, 20001118, 20001124, 20001127, ..., 20061225, 19961225, 20031102, 19971231]

In [20]:
shotDF.SHOT_ZONE_BASIC.unique()

[Mid-Range, In The Paint (Non-RA), Restricted Area, Right Corner 3, Above the Break 3, Left Corner 3, Backcourt]
Categories (7, object): [Mid-Range, In The Paint (Non-RA), Restricted Area, Right Corner 3, Above the Break 3, Left Corner 3, Backcourt]

In [24]:
def proc_date(x):
    strx = str(x)
    year, month = strx[0:4], strx[4:6]
    if int(month) < 9:
        return str(int(year) - 1) + '-' + year[2:]
    else:
        return year + '-' + str(int(year) + 1)[2:]

def proc_zone(x):
    if x == 'Mid-Range':
        return '2 PT (Mid-Range)'
    elif x == 'In The Paint (Non-RA)' or x == 'Restricted Area':
        return '2 PT (Paint)'
    else:
        return '3 PT'

shotDF = (shotDF.assign(SEASON=lambda df: df.GAME_DATE.apply(proc_date),
                        SHOT_ZONE=lambda df: df.SHOT_ZONE_BASIC.apply(proc_zone)
                       )
                .pipe(convert_df))

In [25]:
shotDF.dtypes

GRID_TYPE              category
GAME_ID                category
GAME_EVENT_ID          category
PLAYER_ID              category
PLAYER_NAME            category
TEAM_ID                category
TEAM_NAME              category
PERIOD                 category
MINUTES_REMAINING      category
SECONDS_REMAINING      category
EVENT_TYPE             category
ACTION_TYPE            category
SHOT_TYPE              category
SHOT_ZONE_BASIC        category
SHOT_ZONE_AREA         category
SHOT_ZONE_RANGE        category
SHOT_DISTANCE          category
LOC_X                  category
LOC_Y                  category
SHOT_ATTEMPTED_FLAG    category
SHOT_MADE_FLAG         category
GAME_DATE              category
HTM                    category
VTM                    category
SEASON                 category
SHOT_ZONE              category
dtype: object

In [26]:
shotDF2_crosstab = pd.crosstab(shotDF2.SEASON, shotDF2.SHOT_ZONE).apply(lambda _row: _row/sum(_row),1)
shotDF2_crosstab

SHOT_ZONE,2 PT (Mid-Range),2 PT (Paint),3 PT
SEASON,,,
1996-97,0.399253,0.492887,0.107860
1997-98,0.368495,0.473673,0.157833
1998-99,0.370688,0.461380,0.167932
1999-00,0.382379,0.451204,0.166417
2000-01,0.379246,0.450846,0.169908
2001-02,0.371222,0.447620,0.181158
2002-03,0.369363,0.449245,0.181392
2003-04,0.356459,0.456753,0.186788
2004-05,0.354376,0.449946,0.195678


In [27]:
import pyecharts.options as opts
from pyecharts.faker import Faker
from pyecharts.charts import Line

roundV = np.vectorize(round)
datax = list(map(lambda x: str(x)[2:7], shotDF2_crosstab.index.tolist()))
datay = roundV(shotDF2_crosstab.values.T, 4)

c = (
     Line()
     .add_xaxis(datax)
     .add_yaxis("2 分（中投）", datay[0].tolist())
     .add_yaxis("2 分（禁区）", datay[1].tolist())
     .add_yaxis("3 分", datay[2].tolist())
     .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False)
     )
     .set_global_opts(
         title_opts=opts.TitleOpts(title="NBA 球员投篮选择的变化"),
         xaxis_opts=opts.AxisOpts(
                                  axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
                                  axislabel_opts=opts.LabelOpts(rotate=45, font_size=12, margin=14)
         ),
     )
)

c.render_notebook()

In [1]:
from matplotlib import pyplot as plt
from matplotlib.patches import Arc, Circle, Rectangle, Polygon
import numpy as np

In [2]:
def Arc_fill(center, radius, theta1, theta2, resolution=50, **kwargs):
    # generate the points
    theta = np.linspace(np.radians(theta1), np.radians(theta2), resolution)
    points = np.vstack((radius*np.cos(theta) + center[0], 
                        radius*np.sin(theta) + center[1]))
    # build the polygon and add it to the axes
    poly = Polygon(points.T, closed=True, **kwargs)
    return poly

def draw_ball_field(color='#003370', lw=2):
    # 新建一个大小为(6,6)的绘图窗口
    plt.figure(figsize=(5.36, 5.06), frameon=False)
    # 获得当前的Axes对象ax,进行绘图
    ax = plt.gca(frame_on=False)
    # 设置坐标轴范围
    ax.set_xlim(-268, 268)
    ax.set_ylim(440.5, -65.5)
    # 消除坐标轴刻度
    ax.set_xticks([])
    ax.set_yticks([])
    # 添加备注信息
    # plt.annotate('By xiao F', xy=(100, 160), xytext=(178, 418))
    # 对篮球场进行底色填充
    lines_outer_rec = Rectangle(xy=(-268, -65.5), width=536, height=506,
                                color='#f1f1f1', fill=True, zorder=0)
    # 设置篮球场填充图层为最底层
    # lines_outer_rec.set_zorder(0)
    # 将rec添加进ax
    ax.add_patch(lines_outer_rec)
    # 绘制篮筐,半径为7.5
    circle_ball = Circle(xy=(0, 0), radius=7.5, linewidth=lw, color=color,
                         fill=False, zorder=4)
    # 将circle添加进ax
    ax.add_patch(circle_ball)
    # 绘制限制区
    restricted_arc = Arc(xy=(0, 0), width=80, height=80, theta1=0,
                         theta2=180, linewidth=lw, color=color, 
                         fill=False, zorder=4)
    ax.add_patch(restricted_arc)
    # 绘制篮板,尺寸为(60,1)
    plate = Rectangle(xy=(-30, -7.5), width=60, height=-1, linewidth=lw,
                      color=color, fill=False, zorder=4)
    # 将rec添加进ax
    ax.add_patch(plate)
    # 绘制2分区的外框线,尺寸为(160,190)
    outer_rec_fill = Rectangle(xy=(-80, -47.5), width=160, height=190,
                               linewidth=lw, color="#fefefe", fill=True, zorder=2)
    outer_rec = Rectangle(xy=(-80, -47.5), width=160, height=190,
                          linewidth=lw, color=color, fill=False, zorder=4)
    # 将rec添加进ax
    ax.add_patch(outer_rec_fill)
    ax.add_patch(outer_rec)
    # 绘制罚球站位点
    lane_space_left1 = Rectangle(xy=(-90, 20.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    lane_space_left2 = Rectangle(xy=(-90, 30.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    lane_space_left3 = Rectangle(xy=(-90, 60.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    lane_space_left4 = Rectangle(xy=(-90, 90.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    lane_space_right1 = Rectangle(xy=(80, 20.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    lane_space_right2 = Rectangle(xy=(80, 30.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    lane_space_right3 = Rectangle(xy=(80, 60.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    lane_space_right4 = Rectangle(xy=(80, 90.5), width=10, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    ax.add_patch(lane_space_left1)
    ax.add_patch(lane_space_left2)
    ax.add_patch(lane_space_left3)
    ax.add_patch(lane_space_left4)
    ax.add_patch(lane_space_right1)
    ax.add_patch(lane_space_right2)
    ax.add_patch(lane_space_right3)
    ax.add_patch(lane_space_right4)
    # 绘制罚球区域圆圈,半径为60
    circle_punish1 = Arc(xy=(0, 142.5), width=120, height=120, theta1=0,
                         theta2=180, linewidth=lw, color=color, 
                         fill=False, zorder=4)
    circle_punish2 = Arc(xy=(0, 142.5), width=120, height=120, theta1=180,
                         theta2=360, linewidth=lw, linestyle='--', 
                         color=color, fill=False, zorder=4)
    # circle_punish = Circle(xy=(0, 142.5), radius=60, linewidth=lw,
    #                       color=color, fill=False)
    # 将circle添加进ax
    ax.add_patch(circle_punish1)
    ax.add_patch(circle_punish2)
    # 绘制低位防守区域标志线
    hash_marks_left1 = Rectangle(xy=(-110, -47.5), width=0, height=5,
                                linewidth=lw, color=color,
                                fill=False, zorder=4)
    hash_marks_right1 = Rectangle(xy=(110, -47.5), width=0, height=5,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    hash_marks_left2 = Rectangle(xy=(-50, 82.5), width=5, height=0,
                                linewidth=lw, color=color,
                                fill=False, zorder=4)
    hash_marks_right2 = Rectangle(xy=(45, 82.5), width=5, height=0,
                                 linewidth=lw, color=color,
                                 fill=False, zorder=4)
    ax.add_patch(hash_marks_left1)
    ax.add_patch(hash_marks_right1)
    ax.add_patch(hash_marks_left2)
    ax.add_patch(hash_marks_right2)
    # 绘制三分线的左边线
    three_left_rec_fill = Rectangle(xy=(-220, -47.5), width=440, height=140,
                                    ec="#dfdfdf", fc="#dfdfdf", 
                                    fill=True, zorder=1)
    three_left_rec = Rectangle(xy=(-220, -47.5), width=0, height=140,
                               linewidth=lw, color=color, fill=False, zorder=4)
    # 将rec添加进ax
    ax.add_patch(three_left_rec_fill)
    ax.add_patch(three_left_rec)
    # 绘制三分线的右边线
    three_right_rec = Rectangle(xy=(220, -47.5), width=0, height=140,
                                linewidth=lw, color=color, 
                                fill=False, zorder=4)
    # 将rec添加进ax
    ax.add_patch(three_right_rec)
    # 绘制三分线的圆弧,圆心为(0,0),半径为238.66,起始角度为22.8,结束角度为157.2
    three_arc_fill = Arc_fill(center=(0, 0), radius=238.66, theta1=22.8, 
                              theta2=157.2, resolution=50, linewidth=0,
                              ec="#dfdfdf", fc="#dfdfdf", fill=True, zorder=1)
    three_arc = Arc(xy=(0, 0), width=477.32, height=477.32, theta1=22.8,
                    theta2=157.2, linewidth=lw, color=color,
                    fill=False, zorder=4)
    # 将arc添加进ax
    ax.add_patch(three_arc_fill)
    ax.add_patch(three_arc)
    # 绘制中场标记线
    midcourt_area_marker_left = Rectangle(xy=(-250, 232.5), width=30, height=0,
                                          color=color, linewidth=lw, 
                                          fill=False, zorder=4)
    midcourt_area_marker_right = Rectangle(xy=(220, 232.5), width=30, height=0,
                                           color=color, linewidth=lw,
                                           fill=False, zorder=4)
    ax.add_patch(midcourt_area_marker_left)
    ax.add_patch(midcourt_area_marker_right)
    # 绘制中场处的外半圆,半径为60
    center_outer_arc = Arc(xy=(0, 422.5), width=120, height=120, theta1=180,
                           theta2=0, linewidth=lw, color=color,
                           fill=False, zorder=4)
    # 将arc添加进ax
    ax.add_patch(center_outer_arc)
    # 绘制中场处的内半圆,半径为20
    center_inner_arc = Arc(xy=(0, 422.5), width=40, height=40, theta1=180,
                           theta2=0, linewidth=lw, color=color,
                           fill=False, zorder=4)
    # 将arc添加进ax
    ax.add_patch(center_inner_arc)
    # 绘制篮球场外框线,尺寸为(500,470)
    lines_outer_rec = Rectangle(xy=(-250, -47.5), width=500, height=470,
                                linewidth=lw, color=color,
                                fill=False, zorder=4)
    # 将rec添加进ax
    ax.add_patch(lines_outer_rec)
    return ax